<a href="https://colab.research.google.com/github/nexuslrf/MachineLearning_PY/blob/master/Pytorch_X_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch X Tensorflow

In [1]:
!pip install tensorflow-gpu==2.0.0-beta1

In [2]:
!nvidia-smi

Sat Aug  3 06:26:01 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import tensorflow as tf
import torch
import numpy as np

## Toy Examples:

$y = (3x)^2+2\times(3x)$

->  $y=z^2 +2z; z= 3x$

-> $\frac{\partial y}{\partial z} = 2z + 2; \frac{\partial y}{\partial x} = \frac{\partial y}{\partial z} \frac{\partial z}{\partial x} =3\times(2z+2)$ 

### Pytorch

In [0]:
a = torch.tensor(3.,requires_grad=True)
b = torch.tensor(2.,requires_grad=True)

In [0]:
torch_layer1 = lambda x: a*x
torch_layer2 = lambda z: z**2+b*z

In [0]:
x = torch.tensor(4.,requires_grad=True)
z = torch_layer1(x)
y = torch_layer2(z)

In [0]:
y.backward()

In [20]:
x.grad

tensor(78.)

### Tensorflow

In [0]:
a = tf.Variable(3.0)
b = tf.Variable(2.0)

In [0]:
x = tf.Variable(4.0)

In [0]:
with tf.GradientTape() as tape:
  z = a * x
  y = z**2 + b*z
grad = tape.gradient(y, [x,z])

In [71]:
grad

[<tf.Tensor: id=230, shape=(), dtype=float32, numpy=78.0>,
 <tf.Tensor: id=228, shape=(), dtype=float32, numpy=26.0>]

### Pytorch X Tensorflow

#### TF after torch

In [0]:
a = torch.tensor(3.,requires_grad=True)
b = tf.Variable(2.0)

In [0]:
x = torch.tensor(4.,requires_grad=True)

In [0]:
z = a * x

In [0]:
z_in = tf.Variable(z.data.numpy())

In [0]:
with tf.GradientTape() as tape:
  y = z_in**2 + b*z_in
z_grad = tape.gradient(y,z_in)

In [81]:
z_grad

<tf.Tensor: id=294, shape=(), dtype=float32, numpy=26.0>

In [0]:
z.backward(torch.tensor(z_grad.numpy()))

In [84]:
x.grad

tensor(78.)

#### torch before TF

In [0]:
a = tf.Variable(3.0)
b = torch.tensor(2.,requires_grad=True)

In [0]:
x = tf.Variable(4.)

In [0]:
with tf.GradientTape() as tape:
  z = 3*x

In [0]:
z_in = torch.tensor(z.numpy(), requires_grad=True)

In [0]:
y = z_in**2 + b*z_in

In [100]:
y.backward()
z_grad = z_in.grad.numpy()
z_grad

array(26., dtype=float32)

In [0]:
x_grad = tape.gradient(z,x,z_grad)

In [102]:
x_grad

<tf.Tensor: id=340, shape=(), dtype=float32, numpy=78.0>

## Other Tests

Just ignore~

In [0]:
class torch_layer1x(torch.nn.Module):
  def __init__(self):
    super(torch_layer1x, self).__init__()
    self.a = torch.nn.Parameter(torch.tensor(3.), requires_grad=True)
  
  def forward(self, x):
    return self.a * x

class torch_layer2x(torch.nn.Module):
  def __init__(self):
    super(torch_layer2x, self).__init__()
    self.b = torch.nn.Parameter(torch.tensor(2.), requires_grad=True)
  
  def forward(self, x):
    return x**2 + self.b * x

In [0]:
torch_model = torch.nn.Sequential(
    torch_layer1x(),
    torch_layer2x()
)

In [0]:
x = torch.tensor(4.,requires_grad=True)
y = torch_model(x)

In [0]:
y.backward()

In [38]:
x.grad

tensor(78.)

In [40]:
torch_model[1]

torch_layer2x()

In [46]:
# def hook_backward(module, grad_input, grad_output):
#     print(grad_input)
#     print(grad_output)
    
# def hook_forward(module, input, output):
#     print(input)
#     print(output)

# torch_model[0].register_forward_hook(hook_forward)
# torch_model[0].register_backward_hook(hook_backward)
# torch_model[1].register_forward_hook(hook_forward)
# torch_model[1].register_backward_hook(hook_backward)

In [49]:
x = torch.tensor(4.,requires_grad=False)
y = torch_model(x)

(tensor(4.),)
tensor(12., grad_fn=<MulBackward0>)
(tensor(12., grad_fn=<MulBackward0>),)
tensor(168., grad_fn=<AddBackward0>)
(tensor(12., grad_fn=<MulBackward0>),)
tensor(168., grad_fn=<AddBackward0>)


In [50]:
y.backward()

(tensor(1.), tensor(1.))
(tensor(1.),)
(tensor(1.), tensor(1.))
(tensor(1.),)
(tensor(104.), None)
(tensor(26.),)


In [0]:
# Data
X = np.random.rand(6000,2) * 10 -5
y = 3 * X[:,0]**2 + 5*X[:,1] 

In [0]:
tf_layer = tf.keras.layers.Dense(32, activation='relu')
torch_layer = torch.nn.Linear(32, 1)

In [0]:
Z = tf_layer(X)

In [21]:
Z_in = Z.numpy()
Z_in.shape

(6000, 32)

In [23]:
y_out = torch_layer(torch.Tensor(Z_in))

tensor([[ 0.0774],
        [-0.2586],
        [ 0.0538],
        ...,
        [ 0.1145],
        [-0.1795],
        [ 0.0767]], grad_fn=<AddmmBackward>)